In [ ]:
!pip install biopython scikit-learn --quiet
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import classification_report, accuracy_score
from random import shuffle
def load_sequences(file_paths, window=150, stride=50):
    sequences = []
    labels = []
    for label, path in file_paths.items():
        for record in SeqIO.parse(path, "fasta"):
            seq = str(record.seq).replace("*", "")
            for i in range(0, len(seq) - window + 1, stride):
                frag = seq[i:i+window]
                if len(frag) == window:
                    sequences.append(frag)
                    labels.append(label)
    combined = list(zip(sequences, labels))
    shuffle(combined)
    sequences, labels = zip(*combined)
    return list(sequences), list(labels)
def get_kmers(seq, k=3):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]
def build_kmer_features(sequences, k=3):
    all_kmers = set()
    kmer_counts = []
    for seq in sequences:
        kmers = get_kmers(seq, k)
        count = Counter(kmers)
        kmer_counts.append(count)
        all_kmers.update(count)
    all_kmers = sorted(list(all_kmers))
    df = pd.DataFrame(0, index=np.arange(len(sequences)), columns=all_kmers)
    for i, count in enumerate(kmer_counts):
        for kmer, freq in count.items():
            df.at[i, kmer] = freq
    return df
file_paths = {
    'Zika': '/content/drive/MyDrive/Datasets/Zika_200.fasta',
    'Ebola': '/content/drive/MyDrive/Datasets/Ebola_200.fasta',
    'SARS_CoV_2': '/content/drive/MyDrive/Datasets/SARS_CoV_2_200.fasta',
    'Influenza_A': '/content/drive/MyDrive/Datasets/Influenza_A_200.fasta',
    'Influenza_B': '/content/drive/MyDrive/Datasets/Influenza_B_200.fasta',
    'Tuberculosis': '/content/drive/MyDrive/Datasets/Tuberculosis_200.fasta',
    'Human': '/content/drive/MyDrive/Datasets/Human_200.fasta'
}
data, labels = load_sequences(file_paths, window=150, stride=50)
print(f"\n Augmented sequences: {len(data)}")
X = build_kmer_features(data, k=3)
y = pd.factorize(pd.Series(labels))[0]
selector = SelectKBest(mutual_info_classif, k=300)
X_selected = selector.fit_transform(X, y)
print(X_selected)
selected_indices = selector.get_support(indices=True)
selected_features = X.columns[selected_indices]
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, stratify=y, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
ridge = RidgeClassifier(alpha=1.0)
ridge.fit(X_train_scaled, y_train)
y_pred = ridge.predict(X_test_scaled)
print("\nRidge Classifier Accuracy:", accuracy_score(y_test, y_pred) * 100)
print("\n Classification Report:\n")
print(classification_report(y_test, y_pred))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.1 MB/s eta 0:00:00

 Augmented sequences: 9820
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Ridge Classifier Accuracy: 93.38085539714868

 Classification Report:

              precision    recall  f1-score   support

           0       0.90      0.91      0.91       237
           1       0.93      1.00      0.96       829
           2       0.92      0.85      0.88       426
           3       1.00      0.95      0.97       252
           4       0.94      0.95      0.94       151
           5       0.92      0.72      0.81        67
           6       0.00      0.00      0.00         2

    accuracy                           0.93      1964
   macro avg       0.80      0.77      0.78      1964
weighted avg       0.93      0.93      0.93      1964

